In [1]:
import numpy as np
import gym
import random

In [2]:
def reward_func0(env, field):
    if env.desc[field] == b'G':
        return 1
    else:
        return 0

def reward_func1(env, field):
    if env.desc[field] == b'G':
        return 1
    elif env.desc[field] == b'H':
        return -1
    else:
        return 0

def reward_func1(env, field):
    if env.desc[field] == b'G':
        return 5
    elif env.desc[field] == b'H':
        return -1
    else:
        return 0

In [91]:
env.desc[0]

array([b'S', b'F', b'F', b'F', b'F', b'F', b'F', b'F'], dtype='|S1')

In [14]:
env = gym.make("FrozenLake8x8-v1")
func = reward_func0
total_episodes = 100000
lr = 0.1
gamma = 0.99
n_observations = env.observation_space.n
n_actions = env.action_space.n

qtable = np.zeros((n_observations, n_actions))

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob




steps_per_episode = []
rewards = []


for i in range(total_episodes):
    current_state = env.reset()

    done = False
    total_episode_reward = 0
    steps = 0
    for step in range(MAX_ACTIONS):

        if np.random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(qtable[current_state,:])

        next_state, reward, done, _ = env.step(action)
        # field = divmod(next_state, 8)
        # reward = func(env, field)

        qtable[current_state, action] = qtable[current_state, action] + lr * (reward + gamma * np.max(qtable[next_state, :]) - qtable[current_state, action])
        total_episode_reward = total_episode_reward + reward

        steps += 1

        if done:
            break
        current_state = next_state

    steps_per_episode.append(steps)
    rewards.append(total_episode_reward)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*step) 

print(f"You've reached the goal {sum(rewards)/total_episodes*100}% of the time")
print(f"It usually took {np.mean(steps_per_episode)} steps")


You've reached the goal 0.001% of the time
It usually took 17.69126 steps


In [9]:
MAX_ACTIONS = 20

def train(env, func, total_episodes = 10000, lr = 0.1, gamma = 0.99):

    n_observations = env.observation_space.n
    n_actions = env.action_space.n

    qtable = np.zeros((n_observations, n_actions))

    # Exploration parameters
    epsilon = 1.0                 # Exploration rate
    max_epsilon = 1.0             # Exploration probability at start
    min_epsilon = 0.01            # Minimum exploration probability 
    decay_rate = 0.005            # Exponential decay rate for exploration prob




    steps_per_episode = []
    rewards = []


    for i in range(total_episodes):
        current_state = env.reset()

        done = False
        total_episode_reward = 0
        steps = 0
        for step in range(MAX_ACTIONS):

            if np.random.uniform(0,1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(qtable[current_state,:])

            next_state, reward_default, done, _ = env.step(action)
            field = divmod(next_state, 8)
            reward = func(env, field)

            qtable[current_state, action] = qtable[current_state, action] + lr * (reward + gamma * np.max(qtable[next_state, :]) - qtable[current_state, action])
            total_episode_reward = total_episode_reward + reward_default

            steps += 1

            if done:
                break
            current_state = next_state

        steps_per_episode.append(steps)
        rewards.append(total_episode_reward)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*step) 
        
    return qtable, sum(rewards)/total_episodes*100

def test(env, qtable, attempts = 1000):
    success = 0
    done = False
    current_state = env.reset()

    for _ in range(attempts):
        for _ in range(MAX_ACTIONS):
            action = np.argmax(qtable[current_state,:])
            next_state, reward, done, _ = env.step(action)

            if done:
                success += reward
                break
            current_state = next_state

    return success/attempts*100



In [10]:
env = gym.make("FrozenLake8x8-v1")
qt, succes_rate_train = train(env, reward_func0, total_episodes = 100000)
print(f'train succes rate: {succes_rate_train}%')
succes_rate_test = test(env, qt)
print(f'train succes rate: {succes_rate_test}%')

train succes rate: 0.002%
train succes rate: 0.1%


In [70]:
max_iter = 1000

env = gym.make("FrozenLake8x8-v1")
done = False
current_state = env.reset()

for _ in range(max_iter):
    action = np.argmax(qtable[current_state,:])
    next_state, reward, done, _ = env.step(action)

    if done:
        break
    current_state = next_state

if reward:
    print("Agent succeded")
else:
    print("Agent failed")



Agent succeded
